In [14]:
%load_ext autoreload
%autoreload 2

import sys
if '..' not in sys.path:
    sys.path.append('..')
from notebooks.results import load_results, classification, clusterization, to_latex_table, GremDataFrame
import re
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def extract_letters(s: str) -> str:
    return re.findall(r'[a-zA-Z]+', s)[0]

def fix_dataset_name(s: str) -> str:
    return {
        'TweeterCyberbullying': 'TwitterCyberbullying'
    }.get(s, s)

In [3]:
# results_df = load_results('../results')
results_df = GremDataFrame(load_results('../results/'))
results_df['dataset'] = results_df['dataset'].apply(fix_dataset_name)
results_df['base_head_model'] = results_df['params_name'].apply(extract_letters)
print(len(results_df))
results_df.head()

505


,dataset,datacleaner,vectorizer,params_name,accuracy,f1_score,recall,precision,silhouette,davies_bouldin,calinski_harabasz,bcubed_precission,bcubed_recall,bcubed_f1,base_head_model
0,TwitterCyberbullying,DummyDatacleaner,SpacyMorphTagVectorizer,MLP1,0.915423,0.477922,0.50000,0.457711,NaN,NaN,NaN,NaN,NaN,NaN,MLP
1,TwitterCyberbullying,DummyDatacleaner,SpacyMorphTagVectorizer,MLP2,0.915423,0.477922,0.50000,0.457711,NaN,NaN,NaN,NaN,NaN,NaN,MLP
2,TwitterCyberbullying,DummyDatacleaner,SpacyMorphTagVectorizer,MLP3,0.917413,0.541487,0.53312,0.760302,NaN,NaN,NaN,NaN,NaN,NaN,MLP
3,TwitterCyberbullying,DummyDatacleaner,SpacyMorphTagVectorizer,RandomForest1,0.915423,0.477922,0.50000,0.457711,NaN,NaN,NaN,NaN,NaN,NaN,RandomForest
4,TwitterCyberbullying,DummyDatacleaner,SpacyMorphTagVectorizer,KMeans1.0,NaN,NaN,NaN,NaN,0.086742,3.113713,889.408505,0.845114,0.501334,0.629336,KMeans


In [4]:
results_df['dataset'].unique()

array(['TwitterCyberbullying', 'PrusVsSienkiewicz', 'WritingStyle',
       'Classics5Authors35Books', 'OldNewspapers', 'StarWarsFanfic',
       'EroticVsOthers', 'StarWarsFanficShort'], dtype=object)

In [18]:
pd.options.display.float_format = '{:,.3f}'.format
vecs = list(results_df['vectorizer'].unique())
vecs.remove('HerbertVectorizer')

(
    results_df
    .classification()
    .vectorizer(vecs)
    .sort_values('f1_score', ascending=False)
    .drop_duplicates(subset=['dataset', 'vectorizer'])
    .drop(columns=list(results_df.columns[6:-1]) + ['datacleaner', 'params_name'])
    .pivot(index='vectorizer', columns='dataset', values='f1_score')
)

dataset,Classics5Authors35Books,EroticVsOthers,OldNewspapers,PrusVsSienkiewicz,StarWarsFanfic,StarWarsFanficShort,TwitterCyberbullying,WritingStyle
vectorizer,,,,,,,,
BigramMorphTagVectorizer100,0.861,0.597,0.099,0.743,0.975,0.686,0.550,0.537
BigramMorphTagVectorizer370,0.891,0.593,0.114,0.750,0.841,0.697,0.618,0.544
CountVectorizer1000,0.902,0.645,0.117,0.836,0.975,0.736,0.626,0.657
CountVectorizer5000,0.858,0.657,0.166,0.844,0.975,0.745,0.713,0.672
DPEBPVectorizer100Avg,0.616,0.622,0.149,0.649,0.939,0.762,0.498,0.588
FullMorphTagVectorizer,0.729,0.599,0.119,0.778,0.902,0.721,0.667,0.574
SpacyMorphTagVectorizer,0.737,0.598,0.128,0.832,0.902,0.716,0.541,0.524
TfidfVectorizer1000,0.909,0.645,0.122,0.842,0.951,0.731,0.656,0.666
TfidfVectorizer5000,0.858,0.658,0.165,0.837,1.000,0.745,0.710,0.687


In [8]:
results_df.columns

Index(['dataset', 'datacleaner', 'vectorizer', 'params_name', 'accuracy',
       'f1_score', 'recall', 'precision', 'silhouette', 'davies_bouldin',
       'calinski_harabasz', 'bcubed_precission', 'bcubed_recall', 'bcubed_f1',
       'base_head_model'],
      dtype='object')

In [9]:
to_drop = ['datacleaner', 'params_name', 'accuracy',
       'f1_score', 'recall', 'precision', 'silhouette', 'davies_bouldin',
       'calinski_harabasz', 'bcubed_precission', 'bcubed_recall',]

In [21]:
pd.options.display.float_format = '{:,.3f}'.format
vecs = list(results_df['vectorizer'].unique())
vecs.remove('HerbertVectorizer')

(
    results_df
    .clusterization()
    .vectorizer(vecs)
    .sort_values('bcubed_f1', ascending=False)
    .drop_duplicates(subset=['dataset', 'vectorizer'])
    .drop(columns=to_drop)
    .pivot(index='vectorizer', columns='dataset', values='bcubed_f1')
)

dataset,Classics5Authors35Books,EroticVsOthers,OldNewspapers,PrusVsSienkiewicz,StarWarsFanfic,StarWarsFanficShort,TwitterCyberbullying,WritingStyle
vectorizer,,,,,,,,
BigramMorphTagVectorizer100,0.407,0.658,0.154,0.664,0.657,0.584,0.820,0.680
BigramMorphTagVectorizer370,0.407,0.519,0.158,0.534,0.560,0.628,0.893,0.677
CountVectorizer1000,0.407,0.678,0.160,0.667,0.668,0.618,0.903,0.684
CountVectorizer5000,0.407,0.678,0.158,0.667,0.668,0.667,0.914,0.684
DPEBPVectorizer100Avg,0.409,0.596,0.157,0.624,0.580,0.640,0.907,0.671
FullMorphTagVectorizer,0.407,0.516,0.160,0.568,0.622,0.633,0.903,0.678
SpacyMorphTagVectorizer,0.407,0.678,0.157,0.667,0.665,0.640,0.885,0.684
TfidfVectorizer1000,0.459,0.678,0.160,0.698,0.751,0.616,0.887,0.684
TfidfVectorizer5000,0.531,0.678,0.158,0.986,0.737,0.665,0.913,0.684


In [9]:
(
    results_df
    .dataset(['PrusVsSienkiewicz', 'StarWarsFanfic'])
    .data_cleaner('DummyDatacleaner')
    .vectorizer('SpacyMorphTagVectorizer')
    .classification()
)

,dataset,datacleaner,vectorizer,params_name,accuracy,f1_score,recall,precision,silhouette,davies_bouldin,calinski_harabasz,bcubed_precission,bcubed_recall,bcubed_f1,base_head_model
85,PrusVsSienkiewicz,DummyDatacleaner,SpacyMorphTagVectorizer,RandomForest1,0.852688,0.831615,0.832492,0.830762,NaN,NaN,NaN,NaN,NaN,NaN,RandomForest
86,PrusVsSienkiewicz,DummyDatacleaner,SpacyMorphTagVectorizer,MLP1,0.752688,0.727253,0.737683,0.721667,NaN,NaN,NaN,NaN,NaN,NaN,MLP
87,PrusVsSienkiewicz,DummyDatacleaner,SpacyMorphTagVectorizer,MLP2,0.781720,0.761466,0.776675,0.754386,NaN,NaN,NaN,NaN,NaN,NaN,MLP
88,PrusVsSienkiewicz,DummyDatacleaner,SpacyMorphTagVectorizer,MLP3,0.800000,0.777115,0.785746,0.771183,NaN,NaN,NaN,NaN,NaN,NaN,MLP
311,StarWarsFanfic,DummyDatacleaner,SpacyMorphTagVectorizer,RandomForest1,0.902439,0.901502,0.899821,0.907599,NaN,NaN,NaN,NaN,NaN,NaN,RandomForest
313,StarWarsFanfic,DummyDatacleaner,SpacyMorphTagVectorizer,MLP1,0.853659,0.851449,0.849732,0.862709,NaN,NaN,NaN,NaN,NaN,NaN,MLP
314,StarWarsFanfic,DummyDatacleaner,SpacyMorphTagVectorizer,MLP2,0.865854,0.864218,0.862552,0.872549,NaN,NaN,NaN,NaN,NaN,NaN,MLP
315,StarWarsFanfic,DummyDatacleaner,SpacyMorphTagVectorizer,MLP3,0.890244,0.889438,0.888193,0.893116,NaN,NaN,NaN,NaN,NaN,NaN,MLP


In [8]:
prus_vs_sienkiewicz = results_df[results_df['dataset'] == 'PrusVsSienkiewicz']
data = prus_vs_sienkiewicz.loc[prus_vs_sienkiewicz.groupby('vectorizer')['accuracy'].idxmax()]
data = data[['vectorizer', 'accuracy', 'f1_score', 'base_head_model']].sort_values('f1_score', ascending=False)
print(to_latex_table(
    data,
    place_modifiers='H',
    separate_header=True,
    column_names=['vectorizer', 'accuracy', 'f1 score', 'base head model'],
    float_precission=2,
    caption='Accuracy i miara F1 dla różnych sposobów wektoryzacji powieści Prusa i Sienkiewicza',
    label='prus_vs_sien:vectorizer_comparison',
    bold_labels=['f1_score', 'accuracy']
))

\begin{table}[H]
	\centering
	\begin{tabular}{||c|c|c|c||}
		\hline
		vectorizer & accuracy & f1 score & base head model \\
		\hline\hline
		CountVectorizer5000 & \textbf{0.86} & \textbf{0.84} & RandomForest \\
		TfidfVectorizer1000 & 0.85 & \textbf{0.84} & MLP \\
		TfidfVectorizer5000 & 0.85 & \textbf{0.84} & RandomForest \\
		CountVectorizer1000 & 0.85 & \textbf{0.84} & MLP \\
		SpacyMorphTagVectorizer & 0.85 & 0.83 & RandomForest \\
		FullMorphTagVectorizer & 0.80 & 0.78 & RandomForest \\
		BigramMorphTagVectorizer370 & 0.77 & 0.75 & MLP \\
		BigramMorphTagVectorizer100 & 0.77 & 0.74 & MLP \\
		DPEBPVectorizer100Avg & 0.66 & 0.65 & MLP \\
		\hline
	\end{tabular}
	\caption{Accuracy i miara F1 dla różnych sposobów wektoryzacji powieści Prusa i Sienkiewicza}
	\label{tab:prus_vs_sien:vectorizer_comparison}
\end{table}


In [12]:
df = results_df[results_df['dataset'] == 'OldNewspapers']
data = df.loc[df.groupby('vectorizer')['accuracy'].idxmax()]
data = data[['vectorizer', 'accuracy', 'f1_score', 'base_head_model']].sort_values('f1_score', ascending=False)
print(to_latex_table(
    data,
    place_modifiers='H',
    separate_header=True,
    column_names=['vectorizer', 'accuracy', 'f1 score', 'base head model'],
    float_precission=2,
    caption='Accuracy i miara F1 dla różnych sposobów wektoryzacji artykułów z gazet',
    label='old_newspapers:vectorizer_comparison',
))

{}
\begin{table}[H]
	\centering
	\begin{tabular}{||c|c|c|c||}
		\hline
		vectorizer & accuracy & f1 score & base head model \\
		\hline\hline
		\hline
	\end{tabular}
	\caption{Accuracy i miara F1 dla różnych sposobów wektoryzacji artykułów z gazet}
	\label{tab:old_newspapers:vectorizer_comparison}
\end{table}


In [25]:
df = results_df
data = df.loc[df.groupby('dataset')['accuracy'].idxmax()]
data = data[['vectorizer', 'dataset', 'accuracy', 'f1_score', 'base_head_model']].sort_values('f1_score', ascending=False)
print(to_latex_table(
    data,
    place_modifiers='H',
    separate_header=True,
    column_names=['vectorizer', 'dataset', 'accuracy', 'f1 score', 'base head model'],
    float_precission=2,
    caption='Najlepsze wektoryzery dla każdego ze zbioru danych (zwględem accuracy)',
    label='best_vectorizer:classification',
))

\begin{table}[H]
	\centering
	\begin{tabular}{||c|c|c|c|c||}
		\hline
		vectorizer & dataset & accuracy & f1 score & base head model \\
		\hline\hline
		TfidfVectorizer5000 & StarWarsFanfic & 1.00 & 1.00 & MLP \\
		TfidfVectorizer1000 & Classics5Authors35Books & 0.93 & 0.91 & MLP \\
		CountVectorizer5000 & PrusVsSienkiewicz & 0.86 & 0.84 & RandomForest \\
		TfidfVectorizer5000 & WritingStyle & 0.72 & 0.69 & RandomForest \\
		FullMorphTagVectorizer & TwitterCyberbullying & 0.93 & 0.67 & MLP \\
		CountVectorizer5000 & EroticVsOthers & 0.66 & 0.66 & RandomForest \\
		HerbertVectorizer & OldNewspapers & 0.40 & 0.24 & MLP \\
		\hline
	\end{tabular}
	\caption{Najlepsze wektoryzery dla każdego ze zbioru danych (zwględem accuracy)}
	\label{tab:best_vectorizer:classification}
\end{table}
